In [1]:
import pyscf
import pyscf.tools
from pyscf import lo, gto


from orbitalpartitioning import *


molecule = """
 Fe 1.67785607 0.00052233 0.06475932
 O 0.00000000 0.00000000 -0.47099074
 Fe -1.67785607 -0.00052233 0.06475932
 Cl 1.87002704 -1.09796437 1.99091682
 Cl 2.93244917 -0.98210488 -1.47467288
 Cl 2.37160936 2.07954091 -0.50446591
 Cl -1.87002704 1.09796437 1.99091682
 Cl -2.93244917 0.98210488 -1.47467288
 Cl -2.37160936 -2.07954091 -0.50446591
 """

pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   False,
        spin    =   10, # number of unpaired electrons
        charge  =   -2)
pymol.build()

basis = "def2-svp"
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "minao"
mf.run()
##################################################################




******** <class 'pyscf.scf.rohf.ROHF'> ********
method = ROHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 77  num. singly occ = 10


/Users/nmayhall/opt/anaconda3/envs/pyscf3/lib/python3.10/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


init E= -5289.63018546547
  HOMO = 0.473246078347928  LUMO = 0.476436886667592
cycle= 1 E= -5289.15069324983  delta_E= 0.479  |g|= 2.84  |ddm|= 8.23
  HOMO = -0.000303795913106602  LUMO = 0.0865952448133329
cycle= 2 E= -5276.15095510045  delta_E=   13  |g|= 4.14  |ddm|= 9.58

WARN: HOMO 0.912372601605574 >= LUMO 0.129031757301571

cycle= 3 E= -5291.20851605662  delta_E= -15.1  |g|= 2.39  |ddm|= 9.69
  HOMO = 0.192121528469661  LUMO = 0.454224553216599
cycle= 4 E= -5297.24364203004  delta_E= -6.04  |g|= 1.78  |ddm|= 8.96
  HOMO = 0.193863066010375  LUMO = 0.427954764672703
cycle= 5 E= -5298.42555810989  delta_E= -1.18  |g|= 1.11  |ddm|= 2.29
  HOMO = 0.210765039775295  LUMO = 0.462271180548508
cycle= 6 E= -5298.8540769172  delta_E= -0.429  |g|= 0.769  |ddm|= 0.815
  HOMO = 0.274855837558585  LUMO = 0.489189175123749
cycle= 7 E= -5299.01857415372  delta_E= -0.164  |g|= 0.531  |ddm|= 0.544
  HOMO = 0.368360695612465  LUMO = 0.519055272060588
cycle= 8 E= -5299.07246980463  delta_E= -0.0539

# ROHF/def2-tzvp

In [2]:

pymol.basis = "def2-tzvp"
pymol.build()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "chkfile"
mf.run()
##################################################################




******** <class 'pyscf.scf.rohf.ROHF'> ********
method = ROHF
initial guess = chkfile
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 77  num. singly occ = 10
init E= -5353.88851794802
  HOMO = -0.000750434084710265  LUMO = 0.370972403598101
cycle= 1 E= -5356.91362023608  delta_E= -3.03  |g|= 1.12  |ddm|= 3.58
  HOMO = 0.328631211772801  LUMO = 0.39114743091805
cycle= 2 E= -5356.73650073991  delta_E= 0.177  |g|= 1.56  |ddm|= 1.37
  HOMO = 0.0310683256233876  LUMO = 0.370629567835473
cycle= 3 E= -5357.27786189955  delta_E= -0.541  |g|= 0.155  |ddm|= 0.835
  HOMO = 0.00778525878383041  LUMO = 0.354278233195735
cycle= 4 E= -5357.24992670645  delta_E= 0.0279  |g|= 0.202  |ddm|= 0.414
  HOMO = 0.0148360905669186  LUMO = 0.3

In [3]:

#
#   Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
Cdocc = mf.mo_coeff[:,mf.mo_occ==2]
Csing = mf.mo_coeff[:,mf.mo_occ==1]
Cvirt = mf.mo_coeff[:,mf.mo_occ==0]
ndocc = Cdocc.shape[1]
nsing = Csing.shape[1]
nvirt = Cvirt.shape[1]

print(" Norbs: ndocc = %6i nsing = %6i nvirt = %6i"%(ndocc, nsing, nvirt))

 Norbs: ndocc =     77 nsing =     10 nvirt =    256


In [28]:
# Find AO's corresponding to atoms
full = []
frags = []

f = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 1:
        if ao[2] in ("2p",):
            f.append(ao_idx)
frags.append(f)

f = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 3:
        if ao[2] in ("3p",):
            f.append(ao_idx)
frags.append(f)

f = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 4:
        if ao[2] in ("3p",):
            f.append(ao_idx)
frags.append(f)

f = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 5:
        if ao[2] in ("3p",):
            f.append(ao_idx)
frags.append(f)

f = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 6:
        if ao[2] in ("3p",):
            f.append(ao_idx)
frags.append(f)

f = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 7:
        if ao[2] in ("3p",):
            f.append(ao_idx)
frags.append(f)

f = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 8:
        if ao[2] in ("3p",):
            f.append(ao_idx)
frags.append(f)

full = []
for f in frags:
    for fi in f:
        full.append(fi)

print(frags)
print(full)

[[50, 51, 52], [129, 130, 131], [166, 167, 168], [203, 204, 205], [240, 241, 242], [277, 278, 279], [314, 315, 316]]
[50, 51, 52, 129, 130, 131, 166, 167, 168, 203, 204, 205, 240, 241, 242, 277, 278, 279, 314, 315, 316]


# Form Active Space

In [33]:
#
#   Get full active space
Cenv, Oact, Vact, Venv = dmet_active_space(Cdocc, Cvirt, full, S)

Cact = np.hstack((Oact, Csing, Vact))

#   Localize
Cact = pyscf.lo.PM(pymol).kernel(Cact)

#   Plot 
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)


 Create DMET active space by projecting  333 MOs onto   21 fragment orbitals
Occupied's
   0.56312897
   0.56312749
   0.56310693
   0.56310560
   0.56309504
   0.56309423
   0.56298942
   0.56298805
   0.56297573
   0.56297490
   0.56296860
   0.56296845
   0.56296630
   0.56296290
   0.56296080
   0.56295784
   0.56294143
   0.56294053
   0.53329849
   0.52847289
   0.51962241
Virtuals
   0.83289820
   0.83240135
   0.82962811
   0.82649696
   0.82649584
   0.82648445
   0.82648259
   0.82648120
   0.82647775
   0.82647705
   0.82647446
   0.82647255
   0.82646452
   0.82646204
   0.82645817
   0.82639067
   0.82638426
   0.82638256
   0.82637331
   0.82636916
   0.82634868
 Dmet active space has the following dimensions:
   Environment (occupied)   :    56
   Active (occupied)        :    21
   Active (virtual)         :    21
   Environment (virtual)    :   235


In [37]:
clusters = []

clusters.append([25,28,29,30,31])
clusters.append([22,23,24,26,27])
clusters.append([19,20,21,32,33,34])
clusters.append([41,46,50,6,8,11])
clusters.append([5,10,15,39,43,49])
clusters.append([1,13,18,35,45,52])
clusters.append([2,14,17,36,38,51])
clusters.append([3,7,12,40,44,48])
clusters.append([4,9,16,37,42,47])

order = []
for ci in clusters:
    for cii in ci:
        order.append(cii-1)

Cact_ordered = Cact[:,order]
pyscf.tools.molden.from_mo(mf.mol, "Cact_ordered.molden", Cact_ordered)

init_fspace = []
init_fspace.append((5,0))
init_fspace.append((0,5))
init_fspace.append((3,3))
init_fspace.append((3,3))
init_fspace.append((3,3))
init_fspace.append((3,3))
init_fspace.append((3,3))
init_fspace.append((3,3))
init_fspace.append((3,3))

print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


 init_fspace =  [(5, 0), (0, 5), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3)]
 clusters    =  [[25, 28, 29, 30, 31], [22, 23, 24, 26, 27], [19, 20, 21, 32, 33, 34], [41, 46, 50, 6, 8, 11], [5, 10, 15, 39, 43, 49], [1, 13, 18, 35, 45, 52], [2, 14, 17, 36, 38, 51], [3, 7, 12, 40, 44, 48], [4, 9, 16, 37, 42, 47]]


# Build exchange matrix 

In [ ]:
# Build exchange matrix 
Ka,Kb = mf.get_k()
K = Csing.T@(Ka+Kb)@Cdocc
Ksing_virt = Csing.T@(Ka+Kb)@Cvirt

# Make integrals

In [38]:
#
#   Make integrals
#
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact_ordered.T @ h @ Cact_ordered;
j = Cact_ordered.T @ j @ Cact_ordered;
k = Cact_ordered.T @ k @ Cact_ordered;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact_ordered, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact_ordered @ Cact_ordered.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact_ordered)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa", Cact_ordered.T @ S @ Pa @ S @ Cact_ordered)
np.save("Pb", Cact_ordered.T @ S @ Pb @ S @ Cact_ordered)